In [1]:
# Version 2 of "N_day_SMA_diff_n_forward"
# Changes: allow previous data loaded for sma calculation

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt 
from scipy.stats import ttest_ind
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Define Key Parameters 
max_day_pred = 50
min_day_train = 365
start_date_pred = dt.datetime(2018, 10, 17)

# ticker_name = ['AAPL']

# ticker_name = ['AAPL','DD','HD','JNJ']

ticker_name = ['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 
        'DIS', 'DD', 'XOM', 'GE', 'GS', 'HD', 'IBM', 'INTC', 'JNJ',
        'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV',
        'RTX', 'UHN', 'VZ', 'V', 'WMT']

# # NOTE: UHN doesn't have data after '2018-09-12'
# index_nan = df_close['UHN'].index[df_close['UHN'].apply(np.isnan)]
# # print(index_nan)
ticker_name.remove('UHN')

In [4]:
def downloadData(ticker_name, start_date_pred, sma_length):
    day_sma = sma_length - 1
    d = dt.timedelta(days = day_sma)
    check_num_row = 250 + max_day_pred + day_sma # 250 + 50 + sma_length - 1
    
    day_i = min_day_train + 70 + int(day_sma/5*7) # 365 + 70 + sma_length - 1
    data_row = 20*365 # not expecting more than 20 years of data
    
    end_date = start_date_pred - dt.timedelta(days = 1)
    while data_row != check_num_row:
        start_date = end_date - dt.timedelta(days = day_i)

        data = yf.download(ticker_name, interval = '1d', start = start_date, end = end_date)['Close']
        data_row = data.shape[0]
        
        day_i += (check_num_row - data_row)
        
    print(f"[train (250 days) + test (50 days)] start date: {data.index[0]}; end date: {data.index[-1]}")
    
    return data

In [5]:
def generateSMALen():
    return np.append(np.arange(5, 51, 1), np.arange(100, 250, 50))

In [6]:
def trimData(data, sma_length):
    # trim data rows based on sma_length
    # only need download data once for all sma_length calc. 
    day_sma = sma_length - 1
    check_num_row = 250 + max_day_pred + day_sma # 250 + 50 + sma_length - 1
    
    return data.tail(check_num_row)

In [7]:
def findBestSMAwithGiveN(n_forward, ticker, data_raw):
    # -----------Part 2 Find Best SMA for given n_forward-----------
    # REVISE: load previous data if needed to calculate sma 
    result_per_sma = []

    # Per ticker Per n_forward
    for sma_length in sma_length_list:
        # Adjust data rows 
        data = trimData(data_raw, sma_length)
        # print(data.shape[0]) # 50+250+4 = 304

        # Data manipulation
        # Step 1: calculate SMA and trim data ASAP
        data = data.rename('Close').to_frame()
        data['SMA'] = data['Close'].rolling(sma_length).mean()
        data = data.dropna()

        # Step 2: shift rows for calculating 'Forward Close' and trim data ASAP 
        data['Forward Close'] = data['Close'].shift(-n_forward)
        data = data.dropna()

        # Step 3: calculate 'Forward Return' and set 'Flag'
        data['Forward Return'] = (data['Forward Close'] - data['Close'])/data['Close'] 
        data['flag'] = [int(x) for x in data['Close'] > data['SMA']]
        df = data.dropna()

        # Train and test Model 
        # Step 1: split into train (250 days) and test (50 days)
        train = df.head(250 - n_forward)
        test = df.tail(50 - n_forward)

        # Step 2: extract 'Forward Return' column when flag == 1
        train_forward_returns = train[train['flag'] == 1]['Forward Return']
        test_forward_returns = test[test['flag'] == 1]['Forward Return']

        # Step 3: calculate mean of 'Forward Return'
        train_mean_forward_returns = train_forward_returns.mean()
        test_mean_forward_returns = test_forward_returns.mean()

        # Step 4: caculate p-value based on Welch's t-test (unequal sample size & unequal variance)
        # Verify the average return on test set is statistically similar to
        # to the average return achieved on train set. 
        _, p_value = ttest_ind(train_forward_returns, test_forward_returns, equal_var = False)

        # Save results 
        # Step 1: generate result_per_sma list 
        result_per_sma.append({
            'sma_length': sma_length,
            'train_forward_return': train_mean_forward_returns,
            'test_forward_return': test_mean_forward_returns,
            'p-value': p_value
        })

        # Step 2: sort based on 'train_forward_return' and trim invalid data with low p-value 
        df_result = pd.DataFrame(result_per_sma, columns = ['sma_length', 'train_forward_return', 'test_forward_return', 'p-value'])
        df_result_sorted = df_result.sort_values(by = ['train_forward_return'], ascending = False)
        p_value_threshold = 0.05
        df_result_valid = df_result_sorted[df_result_sorted['p-value'] > p_value_threshold]
        
    return df_result_valid


In [8]:
def findBestSMAperTicker(ticker, sma_length_list):
    # Download data from yfinance
    data_raw = downloadData(ticker, start_date_pred, int(np.amax(sma_length_list)))

    # Set n_forward 
    best_sma_length = []
    n_forward_list = np.arange(2, 50, 1)

    # Find best SMA with given n_forward 
    for n_forward in n_forward_list:
        df_result_valid = findBestSMAwithGiveN(n_forward, ticker, data_raw)

        if df_result_valid.shape[0] != 0:
            best_sma_length.append({'ticker': ticker,
                                    'n_forward': n_forward,
                                    'sma_length': int(df_result_valid.iloc[0]['sma_length']),
                                    'train_forward_return': df_result_valid.iloc[0]['train_forward_return'],
                                    'test_forward_return': df_result_valid.iloc[0]['test_forward_return'],
                                    'p-value': df_result_valid.iloc[0]['p-value']})
        else: 
            best_sma_length.append({'ticker': ticker,
                                    'n_forward': n_forward,
                                    'sma_length': np.nan,
                                    'train_forward_return': np.nan,
                                    'test_forward_return': np.nan,
                                    'p-value': np.nan})

    df_ticker = pd.DataFrame(best_sma_length, 
                             columns = ['n_forward', 'sma_length', 'train_forward_return', 'test_forward_return', 'p-value'])
    df_ticker = df_ticker.set_index('n_forward')
    print(df_ticker)
    
    return df_ticker 

In [9]:
# Main Function
sma_length_list = generateSMALen()
best_sma_length_list = []

for ticker in ticker_name:
    print(f"[TICKER]: {ticker}")
    # Step 1: Get full-size table of best SMA Length for each n_forward days
    df_bestSMA__each_n_forward = findBestSMAperTicker(ticker, sma_length_list)
    
    # Step 2: Find max train_forward_return 
    df_max_train_forward_return = df_bestSMA__each_n_forward.sort_values(by = ['train_forward_return'], ascending = False)
    
    # Step 3: Store values 
    best_sma_length_list.append({'ticker': ticker,
                                 'best_n_forward': df_max_train_forward_return.index[0],
                                 'best_sma_length': df_max_train_forward_return.iloc[0]['sma_length']})
    
    print(f"=================================================\n")

[TICKER]: MMM
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[train (250 days) + test (50 days)] start date: 2016-10-21 00:00:00; end date: 2018-10-15 00:00:00
           sma_length  train_forward_return  test_forward_return   p-value
n_forward                                                                 
2                 7.0              0.002160            -0.003942  0.080649
3                 5.0              0.001870            -0.003378  0.182437
4                50.0              0.002534            -0.004696  0.141594
5                50.0              0.003440            -0.003724  0.184378
6                50.0              0.003978            -0.001895  0.290768
7                

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[train (250 days) + test (50 days)] start date: 2016-10-21 00:00:00; end date: 2018-10-15 00:00:00
           sma_length  train_forward_return  test_forward_return   p-value
n_forward                                                                 
2                 6.0              0.003477             0.002918  0.874650
3                 5.0              0.004360             0.004754  0.932915
4                 5.0              0.005523             0.005116  0.937901
5                 5.0              0.006153             0.004998  0.846460
6                 5.0              0.006075             0.005240  0.898224
7                 5.0          

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[train (250 days) + test (50 days)] start date: 2016-10-21 00:00:00; end date: 2018-10-15 00:00:00
           sma_length  train_forward_return  test_forward_return   p-value
n_forward                                                                 
2                 8.0              0.002979             0.001939  0.807192
3                 8.0              0.005097             0.001958  0.578742
4                 8.0              0.006755             0.003005  0.598570
5                 8.0              0.007533             0.006075  0.865166
6                 8.0              0.009130             0.007564  0.871490
7                 5.0          

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[train (250 days) + test (50 days)] start date: 2016-10-21 00:00:00; end date: 2018-10-15 00:00:00
           sma_length  train_forward_return  test_forward_return   p-value
n_forward                                                                 
2                 5.0              0.003075             0.005144  0.434194
3                 5.0              0.004487             0.007324  0.410269
4                 5.0              0.005543             0.008980  0.390525
5               100.0              0.006905             0.004949  0.698342
6               100.0              0.008639             0.006957  0.764739
7               100.0          

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[train (250 days) + test (50 days)] start date: 2016-10-21 00:00:00; end date: 2018-10-15 00:00:00
           sma_length  train_forward_return  test_forward_return   p-value
n_forward                                                                 
2                42.0              0.001445            -0.003755  0.088818
3                38.0              0.002231            -0.004440  0.057287
4                 5.0              0.003564             0.000653  0.626529
5                31.0              0.002636            -0.005546  0.104617
6                31.0              0.003488            -0.005364  0.117665
7                28.0          

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[train (250 days) + test (50 days)] start date: 2016-10-21 00:00:00; end date: 2018-10-15 00:00:00
           sma_length  train_forward_return  test_forward_return   p-value
n_forward                                                                 
2               200.0              0.003351             0.000187  0.266104
3               200.0              0.005009            -0.000455  0.106343
4               200.0              0.006642            -0.001083  0.057707
5               200.0              0.008202            -0.000394  0.057421
6               200.0              0.009857             0.000205  0.051221
7                44.0          

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[train (250 days) + test (50 days)] start date: 2016-10-21 00:00:00; end date: 2018-10-15 00:00:00
           sma_length  train_forward_return  test_forward_return   p-value
n_forward                                                                 
2                 5.0              0.002246             0.002489  0.928125
3                 5.0              0.002876             0.002551  0.916776
4                 5.0              0.003473             0.002878  0.868373
5                 5.0              0.003622             0.004552  0.808694
6                 5.0              0.003585             0.007140  0.397362
7                 6.0          

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[train (250 days) + test (50 days)] start date: 2016-10-21 00:00:00; end date: 2018-10-15 00:00:00
           sma_length  train_forward_return  test_forward_return   p-value
n_forward                                                                 
2                 5.0              0.000241            -0.003003  0.415267
3                 5.0              0.001009            -0.006768  0.132994
4                47.0             -0.000185            -0.006976  0.167418
5               100.0              0.001069            -0.009131  0.051439
6                47.0              0.001281            -0.006229  0.126569
7                43.0          

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[train (250 days) + test (50 days)] start date: 2016-10-21 00:00:00; end date: 2018-10-15 00:00:00
           sma_length  train_forward_return  test_forward_return   p-value
n_forward                                                                 
2                 5.0              0.001457             0.000383  0.729868
3                 5.0              0.002483             0.001195  0.708320
4                 7.0              0.003398            -0.002394  0.114860
5                 7.0              0.003370            -0.002877  0.133481
6                 6.0              0.003808            -0.002592  0.166339
7                 6.0          

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[train (250 days) + test (50 days)] start date: 2016-10-21 00:00:00; end date: 2018-10-15 00:00:00
           sma_length  train_forward_return  test_forward_return   p-value
n_forward                                                                 
2               100.0              0.002148             0.002291  0.967705
3               100.0              0.003357             0.003301  0.989173
4               100.0              0.004503             0.005014  0.911764
5               100.0              0.005751             0.006815  0.832610
6               100.0              0.006648             0.008255  0.765577
7               100.0          

In [10]:
df_FINAL = pd.DataFrame(best_sma_length_list, 
                             columns = ['ticker', 'best_n_forward', 'best_sma_length'])
df_FINAL = df_FINAL.set_index('ticker')
print(f"\n===============FINAL RESULT===============")
print(df_FINAL)


===============FINAL RESULT===============
        best_n_forward  best_sma_length
ticker                                 
MMM                 33             35.0
AXP                 48            200.0
AAPL                48            200.0
BA                  45             36.0
CAT                 46             23.0
CVX                 34              5.0
CSCO                48             39.0
KO                   5              9.0
DIS                 42             16.0
DD                   9            100.0
XOM                  5              8.0
GE                   2             14.0
GS                   4              6.0
HD                  39             40.0
IBM                 26             40.0
INTC                28             26.0
JNJ                  4              5.0
JPM                  2              6.0
MCD                 46              5.0
MRK                 14             13.0
MSFT                48            200.0
NKE                 29              

In [31]:
# Calculate Total profit returns on test set based on best_n_forward and best_sma_length
test_return_list = []

for ticker in df_FINAL.index:
    best_n_forward = int(df_FINAL.at[ticker, 'best_n_forward'])
    best_sma_length = int(df_FINAL.at[ticker,'best_sma_length'])
    
    # Download data AGAIN from yfinance
    data_raw = downloadData(ticker, start_date_pred, int(np.amax(sma_length_list)))
    
    # Adjust data rows 
    data = trimData(data_raw, best_sma_length)
    
    # Data manipulation
    # Step 1: calculate SMA and trim data ASAP
    data = data.rename('Close').to_frame()
    data['SMA'] = data['Close'].rolling(best_sma_length).mean()
    data = data.dropna()

    # Step 2: shift rows for calculating 'Forward Close' and trim data ASAP 
    data['Forward Close'] = data['Close'].shift(-best_n_forward)
    data = data.dropna()

    # Step 3: calculate 'Forward Return' and set 'Flag'
    data['Forward Return'] = (data['Forward Close'] - data['Close'])
    data['flag'] = [int(x) for x in data['Close'] > data['SMA']]
    
    # Calculate total return 
    data = data.tail(50 - best_n_forward)
    # Step 1: extract 'Forward Return' column when flag == 1
    data_forward_returns = data[data['flag'] == 1]['Forward Return']
    data_forward_returns_close = data[data['flag'] == 1]['Close']
    
    # Step 2: calculate sum of 'Forward Return'
    profit_returns = data_forward_returns.sum() / data_forward_returns_close.sum()

    # Step 3: Store values 
    test_return_list.append({'ticker': ticker, 
                             'best_n_forward': best_n_forward,
                             'best_sma_length': best_sma_length,
                             'profit_return': profit_returns})

df_test_return = pd.DataFrame(test_return_list, 
                             columns = ['ticker', 'best_n_forward', 'best_sma_length','profit_return'])
df_test_return = df_test_return.set_index('ticker')
print(f"\n===============FINAL RESULT===============")
print(df_test_return)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[train (250 days) + test (50 days)] start date: 2016-10-21 00:00:00; end date: 2018-10-15 00:00:00
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[train (250 days) + test (50 days)] start date: 2016-10-21 00:00:00; end date: 2018-10-15 00:00:00
[*********************100%***********************]  1 of 1 completed
[*********************100%*****************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[train (250 days) + test (50 days)] start date: 2016-10-21 00:00:00; end date: 2018-10-15 00:00:00
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[train (250 days) + test (50 days)] start date: 2016-10-21 00:00:00; end date: 2018-10-15 00:00:00
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*****************